In [ ]:
# Define macros
BATCH_SIZE = 4
LABEL_NAMES = ['ljaw', 'ujaw', 'eye']
LABEL_MAP = dict([(y,x) for x,y in enumerate((LABEL_NAMES))])
NUM_KEYPOINTS = 3

In [ ]:
# Install pycocotools
!pip install git+https://github.com/gautamchitnis/cocoapi.git@cocodataset-master#subdirectory=PythonAPI

In [ ]:
# Set device

import tensorflow as tf
import torch

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
GPUs = tf.config.list_physical_devices('GPU')

if len(GPUs) > 0: 
    gpus = tf.config.experimental.list_physical_devices('GPU')
    for gpu in gpus: 
        tf.config.experimental.set_memory_growth(gpu, True)
        print('The following device is configured: ')
        print(gpu)
else:
    print('Warning: No GPUs configured')


In [ ]:
# Create pytorch dataloader

import os
import numpy as np
import torch
import torchvision
import json
from PIL import Image
import random

# In order to be able to use a batch with labels of different sizes (i.e. 2 and 3 salmon bounding boxes)
def collate_fn(batch):
    return tuple(zip(*batch))


class salmonTrainDataset(torch.utils.data.Dataset):
    def __init__(self, root):
        self.root = root # Data main folder
        self.transforms = torchvision.transforms.ToTensor()
        self.imgs = list(sorted(os.listdir(os.path.join(self.root, 'images')))) # Image folder
        self.targets = list(sorted(os.listdir(os.path.join(self.root, 'labels')))) # Label folder

    def __getitem__(self, idx):
        # Input: {keypoints: [[[]]], bboxes: [[]], areas: []}
        # Output: images: (tensor([[[]]]), ...) and
        # targets: ({keypoints: tensor([[[x, y, 1], [x, y, 1], [x, y, 1]], ...]), boxes: tensor([[xmin, ymin, xmax, ymax], ...]), bbox_labels: tensor([]), 
        # labels: tensor([]), image_id: tensor([]), iscrowd: tensor([])}...)
        
        # Import labels and images
        img_path = os.path.join(self.root, 'images', self.imgs[idx])
        label_path = os.path.join(self.root, 'labels', self.targets[idx])
        
        with open(label_path) as f:
            label = json.load(f)
        img = Image.open(img_path)
        
        # To tensor
        target = {}
        target['keypoints'] = torch.as_tensor(label['keypoints'], dtype=torch.float32)
        target['boxes'] = torch.as_tensor(label['bboxes'], dtype=torch.float32)
        target['bbox_labels'] = torch.as_tensor([1]*len(label['bboxes']), dtype=torch.int64)
        target['labels'] = torch.as_tensor([1]*len(label['bboxes']), dtype=torch.int64)
        target['image_id'] = torch.as_tensor([idx])
        #target['area'] = torch.as_tensor(area, dtype=torch.float32)
        target['iscrowd'] = torch.as_tensor([False]*len(label['bboxes']), dtype=torch.int64)
        img_tensor = torchvision.transforms.functional.pil_to_tensor(img)
        img_tensor = torch.div(img_tensor, 255)
        img_tensor = img_tensor.to(device)
        return img_tensor, target

    def __len__(self):
        return len(self.imgs)

In [ ]:
# Get test sample from data loader

dataset = salmonTrainDataset('../input/aug-lab-data-2100-salmon-refined/aug_lab_data')
data_loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
images, targets = next(iter(data_loader))

print(targets[0]['boxes'].shape)


In [ ]:
# visualize data from the data loader
import matplotlib.pyplot as plt
import cv2
import torchvision.transforms as T

def draw_label_on_img(images, targets, ax, batch_size = BATCH_SIZE,  iou_threshold = 0.1):
    toPilImg = T.ToPILImage()
    
    for idx, img in enumerate(images):
        # Transform image tensors to arrays
        img = torch.as_tensor(img.detach().cpu().numpy())
        img = np.array(toPilImg(img).convert('RGB'))
        # Perfroms NMS. Outpur is indices of bounding boxes. If no scores exists, all indices are returned.
        if 'scores' in targets[idx]:
            nms = torchvision.ops.nms(targets[idx]['boxes'], targets[idx]['scores'], iou_threshold = iou_threshold)
            nms = nms.detach().cpu().numpy()
        else:
            nms = list(range(len(targets[idx]['boxes'])))
        
        # Extraxt bounding box array from tensor
        boxes = targets[idx]['boxes'].detach().cpu().numpy()
        
        # Draw all valid bboxes and keypoints onto image
        for i in nms:
            cv2.rectangle(img, 
                tuple(boxes[i][:2].astype(int)),
                tuple(boxes[i][2:].astype(int)), 
                (255,0,0), 5)
            for keypoint in range(NUM_KEYPOINTS):
                color = (255,255,255)
                if keypoint == 0: color = (255,0,0) # ljaw
                if keypoint == 1: color = (0,255,0) # ujaw
                if keypoint == 2: color = (0,0,255) # eye
                cv2.circle(img, 
                           (int(targets[idx]['keypoints'][i][keypoint][0]), int(targets[idx]['keypoints'][i][keypoint][1])), 5,
                           color, 5)
            ax[idx].imshow(img) 
    return ax

# Call visualization function
fig, ax = plt.subplots(ncols=BATCH_SIZE, figsize=(20,20))
ax = draw_label_on_img(images, targets, ax)
plt.show()

In [ ]:
# Define model

from torchvision.models.detection.rpn import AnchorGenerator
anchor_generator = AnchorGenerator(sizes=(32, 64, 128, 256, 512), aspect_ratios=(0.25, 0.5, 0.75, 1.0, 2.0, 3.0, 4.0))
model = torchvision.models.detection.keypointrcnn_resnet50_fpn(pretrained=False,
                                                               pretrained_backbone=True,
                                                               num_keypoints=3,
                                                               num_classes = 2)

In [ ]:
# Review model
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter()
images, targets = next(iter(data_loader))
print(model.roi_heads)

In [ ]:
# Perform and plot untrained forward pass

# Untrained forward pass
images, targets = next(iter(data_loader))
model.to(device).eval()
output = model(images)

# Plot untrained forward pass
fig, ax = plt.subplots(ncols=BATCH_SIZE, figsize=(20,20))
ax = draw_label_on_img(images, output, ax)
plt.show()


In [ ]:
# Import helper functions from pyvision

import sys
sys.path.insert(1, '../input/pytorch-vision/pytorch_vision')

from engine import train_one_epoch, evaluate

In [ ]:
# Train model

model.to(device).train()

# Choose trainable parameters, optimizer and learning rate
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                            step_size=3,
                                            gamma=0.1)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    print("Start an epoch")
    # train for one epoch, printing every 10 iterations
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    # update the learning rate
    lr_scheduler.step()
    torch.save(model.state_dict(), 'salmon_keypoint_detection')

In [ ]:
# Test model (after loading it into input as a dataset)

model.load_state_dict(torch.load('../input/salmon-keypoint-detection-10-epoch/salmon_keypoint_detection_10_epoch', map_location=device))
images, targets = next(iter(data_loader))

model.to(device).eval()
output = model(images)
print(images)
print(output)

fig, ax = plt.subplots(ncols=BATCH_SIZE, figsize=(20,20))
ax = draw_label_on_img(images, output, ax)
plt.show()